# 🎁 Wrappers

Wrappers allow for working with intermediate client-side values.

The values can be:

- re-used in `Nunjucks` values
- retrieved with [`GraphData`](.//Behaviors.ipynb#GraphData)

In [ ]:
if __name__ == "__main__" and "pyodide" in __import__("sys").modules:
    %pip install -q -r requirements.txt

In [ ]:
import ipywidgets as W
from IPython.display import display

import ipyforcegraph.behaviors as B
import ipyforcegraph.behaviors.scales as SC
import ipyforcegraph.graphs as G

## `CaptureAs`

The `CaptureAs` wrapper updates a `column_name` per `node` or `link` provided by the
`wrapped` dynamic variable... which might in turn be wrapped.

> **Note**
>
> _This is similar to the `NodeSelection` and `LinkSelection` values, but can be applied
> to almost anything._
>
> _The order in which values are executed is **non-deterministic**, so care should be
> taken when relying on re-using captured values in `Nunjucks`._

## `ReplaceCssVariables`

The `ReplaceCssVariables` replaces all instances of `var(--{variable name})`. Primarily,
this allows for making stylistic choices base on the theme-dependent variables in
JupyterLab.

> **Note**
>
> _In `ipyforcegraph <0.4.0` this behavior was applied to \_all_ values, but has been
> made opt-in.\_

## `Tint`

`Tint` applies some rule-of-thumb modifications to a color, based on its `value`. This
(and `Colorize`) are explored along with [scales](./Scales.ipynb).

## `Colorize`

`Colorize` can shift colors in in a number of _color spaces_ (such as `rgb` and
`cubehelix`), eeach with a different set of relevant `channels`.

## Example

In [ ]:
ns = B.NodeShapes(
    color=B.CaptureAs(
        "node_color",
        B.ReplaceCssVariables(
            B.CaptureAs(
                "raw_node_color",
                SC.OrdinalColor(
                    "index",
                    domain=[0, 1, 2],
                    range=[
                        f"var(--jp-{i}-color2)" for i in ["brand", "warn", "accent"]
                    ],
                ),
            )
        ),
    )
)
ls = B.LinkShapes(
    color=B.CaptureAs("link_color", B.Nunjucks("{{ link.source.node_color }}"))
)
data = B.GraphData()
fg = G.ForceGraph(
    behaviors=[ns, ls, data],
    background_color=B.ReplaceCssVariables("var(--jp-layout-color1)"),
)
fg.source.nodes = [{"id": "hello"}, {"id": "world"}, {"id": "dolly"}]
fg.source.links = [
    {"source": "hello", "target": "world"},
    {"source": "hello", "target": "dolly"},
]
fg

In [ ]:
out = W.Output()
out

In [ ]:
def show_data(*change):
    src = data.sources[0]
    out.clear_output()
    with out:
        display(src.nodes, src.links)


data.sources[0].observe(show_data)

In [ ]:
data.capturing = True
fg.reheat()